In [43]:
import pandas as pd
import re

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from random import randint
from time import sleep, ctime

from config import driver, username, password, host, port, database
from sqlalchemy import create_engine

def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

In [44]:
# Cities to search
cities = ['Los-Angeles_CA','New-York_NY', 'Chicago_IL', 'Houston_TX']
cities = ['Duarte_CA']

In [45]:
# Start browser
browser = init_browser()



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\kesam\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [46]:
# Create blank lists/dictionaries to store attributes
prices = []
beds = []
baths = []
sizes = []
lot_sizes = []
addresses = []
statuses = []
detail_pages = []

In [47]:
listing_indicators = {}
types = []
fees = []
pricesqfts = []
garages = []
years = []
mls_ids_list = []

## Identify listing attributes from result card

In [48]:
# Loop through each city
for city in cities:

    print(f"Searching {city}...{ctime()}\n")
    
    # Loop through each search result page
    for i in range(1, 3, 1):
        
        # Set dynamic URL
        url = f"https://www.realtor.com/realestateandhomes-search/{city}/pg-{i}"
        browser.visit(url)
        
        print(f"Scraping page {i}...{ctime()}")
        
        # HTML object
        html = browser.html
        
        # Parse HTML with Beautiful Soup
        soup = bs(html, "html.parser")
#         print(soup.prettify())

        # Identify all listings
        listings = soup.find_all('li', attrs={"data-testid": "result-card"})

        # Loop through each listing to identify attributes
        for listing in listings:          
            try:
                price = int(listing.find('span', attrs={"data-label": "pc-price"}).text.strip('$').replace(",",""))
                prices.append(price)
            except:
                prices.append('No Info')
            try:
                bed = int(listing.find('li', attrs={"data-label": "pc-meta-beds"}).text.strip('bed'))
                beds.append(bed)
            except:
                beds.append('No Info')
            try:
                bath = float(listing.find('li', attrs={"data-label": "pc-meta-baths"}).text.strip()[0])
                baths.append(bath)
            except:
                baths.append('No Info')
            try:
                size = listing.find('li', attrs={"data-label": "pc-meta-sqft"}).text.strip('sqft')
                sizes.append(size)
            except:
                sizes.append('No Info')
            try:
                lot_size = listing.find('li', attrs={"data-label": "pc-meta-sqftlot"}).text
                lot_sizes.append(lot_size)
            except:
                lot_sizes.append('No Info')
            try:
                address = listing.find('div', attrs={"data-label": "pc-address"}).text
                addresses.append(address)
            except:
                addresses.append('No Info')
            try:
                status = listing.find('span', attrs={"class": "jsx-3853574337 statusText"}).text
                statuses.append(status)
            except:
                statuses.append('No Info')

            
            # Identify URL to listing detail page
            detail_page = listing.find('a').get('href')
            
            # Append to list
            detail_pages.append(detail_page)
            
            sleep(randint(2,5))
            
        # Generate random number between 2 to 10 seconds to wait before continuing loop
        sleep(randint(2,10))
            
    print("\n----------------------------\n")
    
print(f'Scraping complete...{ctime()}')

Searching Duarte_CA...Thu Sep 30 14:47:29 2021

Scraping page 1...Thu Sep 30 14:47:33 2021
Scraping page 2...Thu Sep 30 14:58:24 2021

----------------------------

Scraping complete...Thu Sep 30 14:59:31 2021


In [49]:
len(detail_pages)

46

## Identify listing attributes from detail page

In [50]:
i = 1
num_page = len(detail_pages)
print(f"Total of {num_page} listings found\n")

# Loop through each listing detail page
for detail_page in detail_pages:
    
    print(f"Scraping details from listing {i} of {num_page}")
    
    # Navigate to each href
    detail_url = f"https://www.realtor.com{detail_page}"
    
    browser.visit(detail_url)
    
    # HTML object
    html = browser.html

    # Parse HTML with Beautiful Soup
    detail_soup = bs(html, "html.parser")

    # Identify all listing-indicators
    details = detail_soup.find('div', attrs={"data-testid": "listing-indicator"})
    re_li = re.compile('rui*')
    
    try:
        for x in details.find_all('li', re_li):
            listing_indicators[x.find_all('div', attrs={'class': re_li})[0].text] = x.find_all('div', attrs={'class': re_li})[1].text
            
        try:
            property_type = listing_indicators['Property Type']
            types.append(property_type)
        except:
            types.append('No Info')
        try:
            hoa_fee = listing_indicators['HOA Fees'].strip('/mo')
            hoa_fee = hoa_fee.strip('$')
            fees.append(hoa_fee)
        except:
            fees.append('No Info')
        try:
            pricesqft = listing_indicators['Price per sqft'].strip('$')
            pricesqfts.append(pricesqft)
        except:
            pricesqfts.append('No Info') 
        try:
            garage = listing_indicators['Garage'].strip(' car')
            garage = garage.strip(' cars')
            garages.append(garage)
        except:
            garages.append('No Info')
        try:
            year = listing_indicators['Year Built']
            years.append(year)
        except:
            years.append('No Info')

    except:
        types.append('No Info')
        fees.append('No Info')
        pricesqfts.append('No Info')
        garages.append('No Info')
        years.append('No Info')
        
        sleep(15)
   
    
    i = i + 1
    
#     Generate random number between 2 to 10 seconds to wait before continuing loop
    sleep(randint(2,10))
    
print(f'\nScraping complete...{ctime()}')

Total of 46 listings found

Scraping details from listing 1 of 46
Scraping details from listing 2 of 46
Scraping details from listing 3 of 46
Scraping details from listing 4 of 46
Scraping details from listing 5 of 46
Scraping details from listing 6 of 46
Scraping details from listing 7 of 46
Scraping details from listing 8 of 46
Scraping details from listing 9 of 46
Scraping details from listing 10 of 46
Scraping details from listing 11 of 46
Scraping details from listing 12 of 46
Scraping details from listing 13 of 46
Scraping details from listing 14 of 46
Scraping details from listing 15 of 46
Scraping details from listing 16 of 46
Scraping details from listing 17 of 46
Scraping details from listing 18 of 46
Scraping details from listing 19 of 46
Scraping details from listing 20 of 46
Scraping details from listing 21 of 46
Scraping details from listing 22 of 46
Scraping details from listing 23 of 46
Scraping details from listing 24 of 46
Scraping details from listing 25 of 46
Scrapi

In [22]:
len(garages)

42

In [51]:
# Add attributes to dataframe
df = pd.DataFrame({'Address': addresses, 'Status': statuses, 'Property Type': types, 'Price': prices, 
                   'Price per sqft': pricesqfts, 'HOA Fees': fees, 'Bed': beds, 'Bath': baths, 'Size': sizes, 
                   'Lot Size': lot_sizes, 'Garage': garages, 'Year Built': years, 'Load DateTime': ctime()})

# Extract address into Street, City, State, Zip
street_city = df['Address'].str.split(',', expand=True)
street_city = street_city.rename(columns={0: 'Street', 1: 'City', 2: 'state_zip'})
state_zip = street_city['state_zip'].str.split(' ', expand=True)
state_zip = state_zip.rename(columns={1: 'State', 2: 'Zip'})
street_city = street_city.drop(columns='state_zip')
state_zip = state_zip.drop(columns=0)

street_city.reset_index(drop=True, inplace=True)
state_zip.reset_index(drop=True, inplace=True)

# Create merged_df
address_df = pd.concat([street_city, state_zip], axis=1) 

merged_df = pd.concat([df, address_df], axis=1) 

In [52]:
merged_df.to_csv('listings.csv', index=False)

In [53]:
browser.quit()

In [54]:
merged_df

,Address,Status,Property Type,Price,Price per sqft,HOA Fees,Bed,Bath,Size,Lot Size,Garage,Year Built,Load DateTime,Street,City,State,Zip
0,"837 Junipero Dr, Duarte, CA 91010",For Sale,No Info,470000,No Info,No Info,2,1.0,832,"6,695sqft lot",No Info,No Info,Thu Sep 30 15:09:48 2021,837 Junipero Dr,Duarte,CA,91010
1,"1020 Bradbourne Ave Spc 47, Duarte, CA 91010",For Sale,Mfd/Mobile,90000,167,No Info,1,1.0,540,No Info,No Info,2000,Thu Sep 30 15:09:48 2021,1020 Bradbourne Ave Spc 47,Duarte,CA,91010
2,"827 Cinnamon Ln Unit 3, Duarte, CA 91010",Contingent,Condo,406000,453,400,2,1.0,896,0.26acre lot,2,1984,Thu Sep 30 15:09:48 2021,827 Cinnamon Ln Unit 3,Duarte,CA,91010
3,"848 Cinnamon Ln Unit 41, Duarte, CA 91010",Pending,Condo,369000,412,400,2,1.0,896,1.96acre lot,2,1984,Thu Sep 30 15:09:48 2021,848 Cinnamon Ln Unit 41,Duarte,CA,91010
4,"1020 Bradbourne Ave Spc 18, Duarte, CA 91010",For Sale,Mfd/Mobile,199900,192,400,2,2.0,"1,040",No Info,2,2021,Thu Sep 30 15:09:48 2021,1020 Bradbourne Ave Spc 18,Duarte,CA,91010
5,"765 El Sur St, Duarte, CA 91010",Pending,Single Family,499250,392,400,3,1.0,"1,275","9,849sqft lot",2,1949,Thu Sep 30 15:09:48 2021,765 El Sur St,Duarte,CA,91010
6,"824 Cinnamon Ln, Duarte, CA 91010",Pending,Condo,418000,421,400,2,2.0,993,1.9acre lot,2,1983,Thu Sep 30 15:09:48 2021,824 Cinnamon Ln,Duarte,CA,91010
7,"830 Cinnamon Ln, Duarte, CA 91010",Contingent,Condo,399000,402,400,2,2.0,993,1.9acre lot,2,1983,Thu Sep 30 15:09:48 2021,830 Cinnamon Ln,Duarte,CA,91010
8,"2415 Flagstone Ave, Duarte, CA 91010",Contingent,Single Family,465000,414,400,3,1.0,"1,122","8,636sqft lot",1,1960,Thu Sep 30 15:09:48 2021,2415 Flagstone Ave,Duarte,CA,91010
9,"1702 1st St, Duarte, CA 91010",For Sale,Multi-Family,750000,599,400,No Info,2.0,"1,252","5,350sqft lot",1,1928,Thu Sep 30 15:09:48 2021,1702 1st St,Duarte,CA,91010


In [55]:
listings_df = merged_df.rename(columns ={"Address":"full_address",
                                        "Status":"status",
                                        "Property Type":"property_type",
                                        "Price":"price",
                                        "Price per sqft":"price_per_sq_ft",
                                        "HOA Fees":"hoa_fees",
                                        "Bed":"bed",
                                        "Bath":"bath",
                                        "Size":"built_area",
                                         "Lot Size":"lot_size",
                                        "Garage":"parking_garage",
                                        "Year Built":"year_built",
                                        "Street":"street_address",
                                        "City":"city",
                                        "State":"state_cd",
                                        "Zip":"zip_code",
                                        "Load DateTime":"load_datetime"})
listings_df

,full_address,status,property_type,price,price_per_sq_ft,hoa_fees,bed,bath,built_area,lot_size,parking_garage,year_built,load_datetime,street_address,city,state_cd,zip_code
0,"837 Junipero Dr, Duarte, CA 91010",For Sale,No Info,470000,No Info,No Info,2,1.0,832,"6,695sqft lot",No Info,No Info,Thu Sep 30 15:09:48 2021,837 Junipero Dr,Duarte,CA,91010
1,"1020 Bradbourne Ave Spc 47, Duarte, CA 91010",For Sale,Mfd/Mobile,90000,167,No Info,1,1.0,540,No Info,No Info,2000,Thu Sep 30 15:09:48 2021,1020 Bradbourne Ave Spc 47,Duarte,CA,91010
2,"827 Cinnamon Ln Unit 3, Duarte, CA 91010",Contingent,Condo,406000,453,400,2,1.0,896,0.26acre lot,2,1984,Thu Sep 30 15:09:48 2021,827 Cinnamon Ln Unit 3,Duarte,CA,91010
3,"848 Cinnamon Ln Unit 41, Duarte, CA 91010",Pending,Condo,369000,412,400,2,1.0,896,1.96acre lot,2,1984,Thu Sep 30 15:09:48 2021,848 Cinnamon Ln Unit 41,Duarte,CA,91010
4,"1020 Bradbourne Ave Spc 18, Duarte, CA 91010",For Sale,Mfd/Mobile,199900,192,400,2,2.0,"1,040",No Info,2,2021,Thu Sep 30 15:09:48 2021,1020 Bradbourne Ave Spc 18,Duarte,CA,91010
5,"765 El Sur St, Duarte, CA 91010",Pending,Single Family,499250,392,400,3,1.0,"1,275","9,849sqft lot",2,1949,Thu Sep 30 15:09:48 2021,765 El Sur St,Duarte,CA,91010
6,"824 Cinnamon Ln, Duarte, CA 91010",Pending,Condo,418000,421,400,2,2.0,993,1.9acre lot,2,1983,Thu Sep 30 15:09:48 2021,824 Cinnamon Ln,Duarte,CA,91010
7,"830 Cinnamon Ln, Duarte, CA 91010",Contingent,Condo,399000,402,400,2,2.0,993,1.9acre lot,2,1983,Thu Sep 30 15:09:48 2021,830 Cinnamon Ln,Duarte,CA,91010
8,"2415 Flagstone Ave, Duarte, CA 91010",Contingent,Single Family,465000,414,400,3,1.0,"1,122","8,636sqft lot",1,1960,Thu Sep 30 15:09:48 2021,2415 Flagstone Ave,Duarte,CA,91010
9,"1702 1st St, Duarte, CA 91010",For Sale,Multi-Family,750000,599,400,No Info,2.0,"1,252","5,350sqft lot",1,1928,Thu Sep 30 15:09:48 2021,1702 1st St,Duarte,CA,91010


In [56]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [57]:
listings_df.to_sql('listings',connection, if_exists='append', index=False)
listings_df.head()

,full_address,status,property_type,price,price_per_sq_ft,hoa_fees,bed,bath,built_area,lot_size,parking_garage,year_built,load_datetime,street_address,city,state_cd,zip_code
0,"837 Junipero Dr, Duarte, CA 91010",For Sale,No Info,470000,No Info,No Info,2,1.0,832,"6,695sqft lot",No Info,No Info,Thu Sep 30 15:09:48 2021,837 Junipero Dr,Duarte,CA,91010
1,"1020 Bradbourne Ave Spc 47, Duarte, CA 91010",For Sale,Mfd/Mobile,90000,167,No Info,1,1.0,540,No Info,No Info,2000,Thu Sep 30 15:09:48 2021,1020 Bradbourne Ave Spc 47,Duarte,CA,91010
2,"827 Cinnamon Ln Unit 3, Duarte, CA 91010",Contingent,Condo,406000,453,400,2,1.0,896,0.26acre lot,2,1984,Thu Sep 30 15:09:48 2021,827 Cinnamon Ln Unit 3,Duarte,CA,91010
3,"848 Cinnamon Ln Unit 41, Duarte, CA 91010",Pending,Condo,369000,412,400,2,1.0,896,1.96acre lot,2,1984,Thu Sep 30 15:09:48 2021,848 Cinnamon Ln Unit 41,Duarte,CA,91010
4,"1020 Bradbourne Ave Spc 18, Duarte, CA 91010",For Sale,Mfd/Mobile,199900,192,400,2,2.0,"1,040",No Info,2,2021,Thu Sep 30 15:09:48 2021,1020 Bradbourne Ave Spc 18,Duarte,CA,91010


In [58]:
new_listings_df = pd.read_sql_table('listings', connection)
new_listings_df

,property_id,full_address,status,property_type,price,price_per_sq_ft,hoa_fees,bed,bath,built_area,parking_garage,lot_size,year_built,street_address,city,state_cd,zip_code,load_datetime
0,1,"837 Junipero Dr, Duarte, CA 91010",For Sale,No Info,470000.0,No Info,No Info,2,1.0,832,No Info,"6,695sqft lot",No Info,837 Junipero Dr,Duarte,CA,91010,2021-09-30 14:32:53
1,2,"1020 Bradbourne Ave Spc 47, Duarte, CA 91010",For Sale,Mfd/Mobile,90000.0,167,No Info,1,1.0,540,No Info,No Info,2000,1020 Bradbourne Ave Spc 47,Duarte,CA,91010,2021-09-30 14:32:53
2,3,"827 Cinnamon Ln Unit 3, Duarte, CA 91010",Contingent,Condo,406000.0,453,400,2,1.0,896,2,0.26acre lot,1984,827 Cinnamon Ln Unit 3,Duarte,CA,91010,2021-09-30 14:32:53
3,4,"848 Cinnamon Ln Unit 41, Duarte, CA 91010",Pending,Condo,369000.0,412,400,2,1.0,896,2,1.96acre lot,1984,848 Cinnamon Ln Unit 41,Duarte,CA,91010,2021-09-30 14:32:53
4,5,"1020 Bradbourne Ave Spc 18, Duarte, CA 91010",For Sale,Mfd/Mobile,199900.0,192,400,2,2.0,"1,040",2,No Info,2021,1020 Bradbourne Ave Spc 18,Duarte,CA,91010,2021-09-30 14:32:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,"2333 Bloomdale St, Duarte, CA 91010",Contingent,Single Family,599000.0,701,180,2,1.0,855,2,"5,050sqft lot",1950,2333 Bloomdale St,Duarte,CA,91010,2021-09-30 15:09:48
84,85,"Opal Cyn, Duarte, CA 91010",Contingent,Land,399000.0,701,180,No Info,No Info,No Info,2,34.06acre lot,1950,Opal Cyn,Duarte,CA,91010,2021-09-30 15:09:48
85,86,"1820 Huntington Dr, Duarte, CA 91010",Pending,Condo,475000.0,448,412,2,2.0,"1,060",2,0.75acre lot,1985,1820 Huntington Dr,Duarte,CA,91010,2021-09-30 15:09:48
86,87,"2074 Goodall Ave, Duarte, CA 91010",Pending,Single Family,545000.0,588,412,3,1.0,927,1,"6,802sqft lot",1952,2074 Goodall Ave,Duarte,CA,91010,2021-09-30 15:09:48
